# Visualizacion del Coronavirus (COVID19) Mundial
por: Jose R. Zapata - https://joserzapata.github.io/



He visto en las redes sociales varias visualizaciones de los datos del COVID 19 y queria realizarlos en Python para tener la actualizacion de las graficas
actualizadas cada dia, y ademas practicar el uso de [plotly](https://plot.ly/) para visualizacion interactiva.

Las Graficas se actualizaran diariamente con los nuevos datos!

Informacion extraida de 2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE

https://github.com/CSSEGISandData/COVID-19

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

!pip install chart_studio -q
import chart_studio

In [ ]:
#chart-studio api
username = '' # your username
api_key = '' # your api api_key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
import chart_studio.plotly as py

In [ ]:
# Leer datos
confirmed = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
recovered = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')
death = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

## Datos CSSEGISandData/COVID-19

Descripcion de los datos

**Province/State:** China - province name; US/Canada/Australia/ - city name, state/province name; Others - name of the event (e.g., "Diamond Princess" cruise ship); other countries - blank.

**Country/Region:** country/region name conforming to WHO (will be updated).

**Last Update:** MM/DD/YYYY HH:mm (24 hour format, in UTC).

**Confirmed: **the number of confirmed cases. For Hubei Province: from Feb 13 (GMT +8), we report both clinically diagnosed and lab-confirmed cases. For lab-confirmed cases only (Before Feb 17), please refer to who_covid_19_situation_reports. For Italy, diagnosis standard might be changed since Feb 27 to "slow the growth of new case numbers."

**Deaths:** the number of deaths.
**Recovered:** the number of recovered cases.

In [23]:
confirmed.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
0,Thailand,2,3,5,7,8,8,14,14,14,19,19,19,19,25,25,25,25,32,32,32,33,33,33,33,33,34,35,35,35,35,35,35,35,35,37,40,40,41,42,42,43,43,43,47,48,50,50,50,53,59,70,75,82,114,147,177,212,272
1,Japan,2,1,2,2,4,4,7,7,11,15,20,20,20,22,22,45,25,25,26,26,26,28,28,29,43,59,66,74,84,94,105,122,147,159,170,189,214,228,241,256,274,293,331,360,420,461,502,511,581,639,639,701,773,839,825,878,889,924
2,Singapore,0,1,3,3,4,5,7,7,10,13,16,18,18,24,28,28,30,33,40,45,47,50,58,67,72,75,77,81,84,84,85,85,89,89,91,93,93,93,102,106,108,110,110,117,130,138,150,150,160,178,178,200,212,226,243,266,313,345
3,Nepal,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,Malaysia,0,0,0,3,4,4,4,7,8,8,8,8,8,10,12,12,12,16,16,18,18,18,19,19,22,22,22,22,22,22,22,22,22,22,22,22,23,23,25,29,29,36,50,50,83,93,99,117,129,149,149,197,238,428,566,673,790,900


In [ ]:
#recovered.head()

In [ ]:
#death.head()

## Datos Generales de cada Dataframe

In [ ]:
#confirmed.info()

In [ ]:
#recovered.info()

In [ ]:
#death.info()

### Eliminar Ubicacion
Se va realizar un analisis general de los datos y No se van a tomar los datos geograficos de *latitud*, *longitud* y los datos de *Province/State* estan incompletos.

Solo se realizara un analisis por pais entonces se eliminaran las columnas mencionadas anteriormente


In [ ]:
confirmed = confirmed.drop(columns=['Lat', 'Long','Province/State'])

In [ ]:
recovered = recovered.drop(columns=['Lat', 'Long','Province/State'])

In [ ]:
death = death.drop(columns=['Lat', 'Long','Province/State'])

### Consolidar datos

In [ ]:
confirmed_group = confirmed.groupby(by='Country/Region')
confirmed_group = confirmed_group.aggregate(np.sum)
confirmed_group = confirmed_group.T
confirmed_group.index.name = 'date'
confirmed_group =  confirmed_group.reset_index()

In [ ]:
recovered_group = recovered.groupby(by='Country/Region')
recovered_group = recovered_group.aggregate(np.sum)
recovered_group = recovered_group.T
recovered_group.index.name = 'date'
recovered_group =  recovered_group.reset_index()

In [ ]:
death_group = death.groupby(by='Country/Region')
death_group = death_group.aggregate(np.sum)
death_group = death_group.T
death_group.index.name = 'date'
death_group =  death_group.reset_index()

In [ ]:
confirmed_melt = confirmed_group.melt(id_vars="date")
confirmed_melt.rename(columns = {'value':'Confirmados', 'date':'Fecha'}, inplace = True)

### Datos Mundiales

In [ ]:
# Numero de Casos confirmados por dia en el mundo
column_names = ["Fecha", "Confirmados", "Recuperados","Muertos"]
world = pd.DataFrame(columns = column_names)
world['Fecha'] = confirmed_group['date']
world['Confirmados'] = confirmed_group.iloc[:,1:].sum(1)
world['Recuperados'] = recovered_group.iloc[:,1:].sum(1)
world['Muertos'] = death_group.iloc[:,1:].sum(1)

In [ ]:
#world.tail()

# Visualizacion con Plotly

## Valores Mundiales de Confirmados, Recuperados y Muertos

In [18]:
temp = pd.DataFrame(world.iloc[-1,:]).T
tm = temp.melt(id_vars="Fecha", value_vars=[ "Confirmados", "Recuperados","Muertos"])
fig = px.bar(tm, x="variable" , y="value", color= 'variable',
             color_discrete_sequence=["blue", "green", "red"],
             height=400, width=600,
             title= f'Total de Casos Mundiales de COVID 19 - {str(world.iloc[-1,0])}')

# Agregar el valor de cada columna en la parte superior
for n in range(len(tm['variable'])):
  fig.add_annotation(x=tm.iloc[n,1], y=tm.iloc[n,2], text=str(tm.iloc[n,2]),
                     showarrow=True, ax=0, ay=-30)
fig.layout.update(showlegend=False,
                  yaxis =  {"title": {"text": "Numero de Personas"}}, # Cambiar texto eje y
                  xaxis =  {"title": {"text": ""}} #Esconder nombre eje x
                  )
# grabar grafica en chart-studio
#py.plot(fig, filename = 'total_casos_general', auto_open=True)
fig.show()

## Progresion Mundial en el Tiempo de de Confirmados, Recuperados y Muertos

In [19]:
world_melt = world.melt(id_vars='Fecha', value_vars= list(world.columns)[1:], var_name=None)

fig = px.line(world_melt, x="Fecha", y= 'value',
              color='variable',
              title=f'Total de Casos en el tiempo de COVID 19 - {world.iloc[-1,0]}')
for n in list(world.columns)[1:]:
  fig.add_annotation(x=world.iloc[-1,0], y=world.loc[world.index[-1],n],
                     text=n,
                     showarrow=True, ax=-50, ay=-20)
fig.layout.update(showlegend = False,
                  yaxis =  {"title": {"text": "Numero de Personas"}}, # Cambiar texto eje y
                  )
# grabar grafica en chart-studio
#py.plot(fig, filename = 'total_casos_serie', auto_open=True)
fig.show()

## Total Casos Confirmados de COVID 19 por Pais (Excluyendo China)

In [20]:
df1 = confirmed_group.drop(columns=["China"])
# Cambiar el nombre de la columna
df1.rename(columns = {'date':'Fecha'}, inplace = True) 
df_melt = df1.melt(id_vars='Fecha', value_vars= list(df1.columns)[1:], var_name=None)
fig = px.line(df_melt, x='Fecha' , y='value', color='Country/Region',
              color_discrete_sequence=px.colors.qualitative.G10,
              title=f'Total Casos Confirmados de COVID 19 por Pais (Excluyendo China) - {world.iloc[-1,0]}')
# 8 paises mas infectados
fecha = df1['Fecha'].iloc[-1] #obtener la fecha del ultimo dato
paises = df1.iloc[-1,1:] #obtener la serie sin el primer dato, fecha
paises.sort_values(ascending=False, inplace=True)
mas_infectados=[]
for n in range(8):
  fig.add_annotation(x=fecha, y=paises[n], text=paises.index[n],
                     showarrow=True, ax=+30, ay=0)
  mas_infectados.append(paises.index[n])
fig.layout.update(showlegend=False,
                  yaxis =  {"title": {"text": "Numero de Personas"}}, # Cambiar texto eje y
                  )
# grabar grafica en chart-studio
#py.plot(fig, filename = 'total_casos_no_china', auto_open=True)
fig.show()

## Total Casos Confirmados de COVID 19 por Pais (Excluyendo los 8 mas infectados y China)

In [21]:
df2 = confirmed_group.drop(columns=["China"] +mas_infectados)
# Cambiar el nombre de la columna
df2.rename(columns = {'date':'Fecha'}, inplace = True) 

df_melt2 = df2.melt(id_vars='Fecha', value_vars= list(df2.columns)[1:], var_name=None)
fig = px.line(df_melt2, x='Fecha' , y='value', color='Country/Region',
              color_discrete_sequence=px.colors.qualitative.G10,
              title=f'Total Casos Confirmados de COVID 19 por Pais (Excluyendo los 8 mas infectados) - {world.iloc[-1,0]}')

fecha = df2['Fecha'].iloc[-1] #obtener la fecha del ultimo dato
paises = df2.iloc[-1,1:] #obtener la serie sin el primer dato, fecha
paises.sort_values(ascending=False, inplace=True)
for n in range(8):
  fig.add_annotation(x=fecha, y=paises[n], text=paises.index[n],
                     showarrow=True, ax=+30, ay=0)
fig.layout.update(showlegend=False,
                  yaxis =  {"title": {"text": "Numero de Personas"}}, # Cambiar texto eje y
                  )
#py.plot(fig, filename = 'total_casos_no_8_infectados', auto_open=True)
fig.show()

## Animacion del Mapa de Evolucion Temporal del Codiv 19

In [23]:
confirmed_melt['Fecha'] = pd.to_datetime(confirmed_melt['Fecha'])
confirmed_melt['Fecha'] = confirmed_melt['Fecha'].dt.strftime('%m/%d/%Y')
confirmed_melt['size'] = confirmed_melt['Confirmados'].pow(0.3)

fig = px.scatter_geo(confirmed_melt, locations="Country/Region", locationmode='country names', 
                     color="Confirmados", size='size', hover_name="Country/Region", 
                     range_color= [0, max(confirmed_melt['Confirmados'])+2], 
                     projection="natural earth", animation_frame="Fecha", 
                     title='Contagiados COVID 19 en el Tiempo')
fig.update(layout_coloraxis_showscale=False)
py.plot(fig, filename = 'mapa_evolucion_temporal', auto_open=True)
fig.show()

## Numero de Casos Confirmados en Colombia

In [22]:
fig = px.line(confirmed_group,x='date' , y='Colombia',
              title=f'Total Casos Confirmados de COVID 19 Colombia - {world.iloc[-1,0]}')
fig.layout.update(showlegend=False,
                  yaxis =  {"title": {"text": "Numero de Personas"}}, # Cambiar texto eje y
                  xaxis =  {"title": {"text": "Fecha"}})
#py.plot(fig, filename = 'Colombia_general', auto_open=True)
fig.show()

# Refencias

- https://github.com/CSSEGISandData/COVID-19
- https://www.kaggle.com/imdevskp/covid-19-analysis-viz-prediction-comparisons
https://junye0798.com/post/build-a-dashboard-to-track-the-spread-of-coronavirus-using-dash/
-https://github.com/Perishleaf/data-visualisation-scripts/tree/master/dash-2019-coronavirus
- https://medium.com/tomas-pueyo/coronavirus-por-qu%C3%A9-debemos-actuar-ya-93079c61e200
